In [26]:
import os, os.path, re
from astropy.wcs import WCS
from astropy.io import fits
import astropy.units as u
import numpy as np
from astropy.visualization import (MinMaxInterval, SqrtStretch,
                                   ImageNormalize)
from astropy.visualization.wcsaxes import add_beam
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse, Circle
from astropy.coordinates import SkyCoord

burst = (-58.123495386, -20.597057565) # 2295 in 240s
src2 = (-58.118828096, -20.596769124)

b = SkyCoord(burst[0]*u.deg, burst[1]*u.deg)
s2 = SkyCoord(src2[0]*u.deg, src2[1]*u.deg)
sep = b.separation(s2)
print(sep)


0d00m15.76245943s


In [27]:

lband_images = ["obs-l2-qc2/im4htc-cad1/cubes/cube-1-240s.fits",
                "obs-l1-qc2/im3/im3-MFS-I-image.fits", 
                "obs-l2-qc2/im3/im3-MFS-I-image.fits", 
                "obs-l4-qc2/im3/im3-MFS-I-image.fits"]
uhf_images = ["obs-u2-qc2/im3/im3-MFS-I-image.fits", 
                "obs-u3-qc2/im3-MFS-image.fits"] 
lband_images2 = ["obs-l2-qc2/im4htc-cad1/cubes/cube-1-240s.fits",
                "obs-l1-qc2/im3/im3-MFS-V-image.fits", 
                "obs-l2-qc2/im3/im3-MFS-V-image.fits", 
                "obs-l4-qc2/im3/im3-MFS-V-image.fits",
                "",
                "obs-l2-qc2/im4htc-cad1/cubes/psf-cube-1-240s.fits"
                ]


In [28]:
def plot_burst_row(images, norms, regions=[], unitsize=2, radius=1, filename=None):
    hdus = [fits.open(im)[0] if im else None for im in images]
    headers = [hdu.header if hdu else None for hdu in hdus]
    ra0, dec0 = burst

    # get pixel position of centre in original WCS
    def get_box(hdr, ra0, dec0):
        wcs = WCS(hdr)
        wcs = wcs.dropaxis(2).dropaxis(2)  # drop FREQ/STOKES axes
        x0, y0 = wcs.world_to_pixel_values(ra0, dec0)
        dx = int(radius / 60 / hdr['CDELT1']) 
        # convert markup to pixel coordinates
        regs = [(wcs.world_to_pixel_values(ra, dec), color, r, ls) for (ra, dec), color, r, ls in regions]
        return x0, y0, dx, regs

    # create new fake WCS centred on x0, y0
    def make_fake_wcs(hdr, x0, y0):
        hdr['CRPIX1'] = x0+1 
        hdr['CRPIX2'] = y0+1 
        hdr['CRVAL1'] = hdr['CRVAL2'] = 0
        hdr['CTYPE1'] = "LON"
        hdr['CTYPE2'] = "LAT"
        hdr['CDELT1'] *= 60
        hdr['CDELT2'] *= 60
        wcs = WCS(hdr)
        return wcs.dropaxis(2).dropaxis(2)
    
    boxes = {}
    for i, hdu in enumerate(hdus):
        boxes[i] = get_box(hdu.header, ra0, dec0) if hdu else None
    
    # fig, axes = plt.subplots(1, len(images), figsize=(unitsize*(len(images)), unitsize*1.5), sharey=True, 
    #                          layout='compressed',
    #                          subplot_kw=dict(projection=wcs))
    figsize = (unitsize*6, unitsize*2.5)
    fig = plt.figure(figsize=figsize, layout='compressed', dpi=600)
    axes = [None]*len(images)
    
    img = [None] * len(images)
    for i, _ in enumerate(images):
        if hdus[i] is None:
            continue
        # select slice
        if i == 0:
            slc = (0, 1402)
            norm = norms[0]
        elif i == 6:
            slc = (1, 1402) 
            norm = norms[0]
        elif i == 11:
            slc = (0, 1402)
            norm = norms[3]
        else:
            slc = (0,0)
            norm = norms[2] if (i%6)<4 else norms[2]
        # select boxes
        hdr = headers[i]
        if i == 11:  # move PSF to burst position
            hdr['CRVAL1'] = ra0
            hdr['CRVAL2'] = dec0
        x0, y0, r, regs = boxes[i] = get_box(hdr, ra0, dec0)
        wcs = make_fake_wcs(hdr, x0, y0)
        axes[i] = plt.subplot(2, 6, 1+i, projection=wcs)

        img[i] = axes[i].imshow(hdus[i].data[slc]*1e+6, origin='lower',  
                             cmap='inferno', norm=norm)
        if i % 6:
            axes[i].coords[1].set_axislabel('')
            axes[i].coords[1].set_ticklabel_visible(False)
        else:
            axes[i].coords[1].set_axislabel('$\Delta$Dec, arcmin')
        axes[i].coords[0].set_axislabel('$\Delta$RA, arcmin') # ,minpad=-2)
        axes[i].coords[0].set_ticks_position('b')
        axes[i].coords[1].set_ticks_position('l')
        axes[i].set_xlim([x0+r, x0-r])
        axes[i].set_ylim([y0+r, y0-r])
        # add markup
        for (x, y), color, r, ls in regs:
            x1, y1 = wcs.pixel_to_world_values(x, y)
            r1 = r/60  # radius is in arcsec
            reg = Circle((x1, y1), radius=r1, 
                            transform=axes[i].get_transform('world'), facecolor='none', 
                            edgecolor=color, ls=ls, lw=1, zorder=1000)
            axes[i].add_patch(reg)
        if i != 11:
        # add beam
            if i not in (0, 6):
                bmaj, bmin, bpa = hdr['BMAJ'], hdr['BMIN'], hdr["BPA"]
            else:
                bmaj, bmin, bpa = 2.937e-3, 1.823e-3, 153
            beam = Ellipse((-radius*0.8, radius*0.8), height=bmaj*60, width=bmin*60, angle=-bpa,
                        transform=axes[i].get_transform('world'), facecolor='cyan', edgecolor='none', lw=0, zorder=1000)  
            axes[i].add_patch(beam)      

    cbar1 = fig.colorbar(img[0], ax=axes[0], orientation='horizontal', shrink=1, location="top")
    cbar1.set_label('$\mu$Jy/beam')
    cbar2 = fig.colorbar(img[1], ax=axes[1:4], orientation='horizontal', shrink=.8, aspect=60*0.8, location="top")
    cbar2.set_label('$\mu$Jy/beam')
    cbar3 = fig.colorbar(img[4], ax=axes[4:6], orientation='horizontal', shrink=.8, aspect=40*0.8, location="top")
    cbar3.set_label('$\mu$Jy/beam')

    if filename:
        fig.savefig(filename, bbox_inches='tight', pad_inches=0, dpi=600)

In [29]:
def plot_image_row(images, vmin=-10, vmax=50, unitsize=2, radius=1):
    hdus = [fits.open(im)[0] for im in images]
    hdr = hdus[0].header
    radius /= 60
    ra0, dec0 = burst

    # get pixel position of centre in original WCS
    wcs = WCS(hdr)
    wcs = wcs.dropaxis(2).dropaxis(2)  # drop FREQ/STOKES axes
    x0, y0 = wcs.world_to_pixel_values(ra0, dec0)
    dx = dy = int(radius / hdr['CDELT1']) 

    # create new fake WCS centred on x0, y0
    hdr['CRPIX1'] = x0+1 
    hdr['CRPIX2'] = y0+1 
    hdr['CRVAL1'] = hdr['CRVAL2'] = 0
    hdr['CTYPE1'] = "LON"
    hdr['CTYPE2'] = "LAT"
    hdr['CDELT1'] *= 60
    hdr['CDELT2'] *= 60

    wcs = WCS(hdr)
    wcs = wcs.dropaxis(2).dropaxis(2)

    fig, axes = plt.subplots(1, len(images), figsize=(unitsize*(len(images)+1.1), unitsize*1.5), sharey=True, 
                             layout='compressed', 
                             subplot_kw=dict(projection=wcs))

    norm = ImageNormalize(vmin=vmin, vmax=vmax) # , stretch=SqrtStretch())

    for i, _ in enumerate(images):
        hdr = hdus[i].header
        img = axes[i].imshow(hdus[i].data[0,0]*1e+6,  
                             cmap='inferno', norm=norm)
        if i > 0:
            axes[i].coords[1].set_axislabel('')
            axes[i].coords[1].set_ticklabel_visible(False)
        else:
            axes[i].coords[1].set_axislabel('$\Delta$Dec, arcmin')
        axes[i].coords[0].set_axislabel('$\Delta$Dec, arcmin') # ,minpad=-2)
        #axes[i].coords[1].set_axislabel(' ',minpad=-2)
        axes[i].coords[0].set_ticks_position('b')
        axes[i].coords[1].set_ticks_position('l')
        axes[i].set_xlim([x0+dx, x0-dx])
        axes[i].set_ylim([y0-dy, y0+dy])
        #add_beam(axes[i], major=hdr['BMAJ']*60, minor=hdr['BMIN']*60, angle=hdr['BPA'])
        #axes[i].set_xlabel('')
        #axes[i].coords[0].set_ticks(np.arange(-3,3.1)*u.arcmin)
        #axes[i].coords[0].set_ticklabel(["{int(l)}''" for l in np.arange(-3,3.1)])
        #axes[i].set_xticks(np.arange(-3,3.1), labels=["{int(l)}''" for l in np.arange(-3,3.1)])
#     cbar = fig.colorbar(img, ax=axes, orientation='vertical', shrink=1)
#     cbar.set_label('$\mu$Jy/beam')
    cbar1 = fig.colorbar(img, ax=axes[0,4], orientation='horizontal', shrink=1, location="top")
    cbar1.set_label('$\mu$Jy/beam')
    cbar2 = fig.colorbar(img, ax=axes[1:5], orientation='horizontal', shrink=.8, aspect=40*0.8, location="top")
    cbar2.set_label('$\mu$Jy/beam')

#fig.set_tight_layout(True)
# plt.grid(color='white', ls='solid')

In [ ]:

l1norm = ImageNormalize(vmin=-10, vmax=100) #, stretch=SqrtStretch())
lnorm = ImageNormalize(vmin=-10, vmax=50) # , stretch=SqrtStretch())
unorm = ImageNormalize(vmin=-10, vmax=100) # , stretch=SqrtStretch())
psfnorm = ImageNormalize(vmin=-10/262*1e+6, vmax=100/262*1e+6) # , stretch=SqrtStretch())
regions = [(burst, 'yellow', 8, '-'), (src2, 'yellow', 8, ':')]

plot_burst_row(lband_images+uhf_images+lband_images2, 
               radius=1, unitsize=2.2, 
               norms=[l1norm, lnorm, unorm, psfnorm], 
               regions=regions, filename='l2-cutouts.png')
